In [42]:
import nest_asyncio
nest_asyncio.apply()

In [43]:
from dotenv import load_dotenv
load_dotenv()

True

In [44]:
from pprint import pprint

#### Defininng colang content and yaml content
* This can be done using files as well. First we will perform with text in cells and go forward

In [45]:
colang_content= """
define user express greeting
    "hello"
    "hi"
    "yo"

define bot express greeting
    "Hello there!! How can i help you today buddy?"

define flow hello
    user express greeting
    bot express greeting
"""

yaml_content="""
models:
- type: main
  engine: openai
  model: gpt-3.5-turbo


"""

from nemoguardrails import LLMRails, RailsConfig
config= RailsConfig.from_content(
    yaml_content= yaml_content,
    colang_content=colang_content
)

In [46]:
rails= LLMRails(config= config)

* We can use rails.generate as well instead of rails.genrate_async() for asynchronisation

In [47]:
res= await rails.generate_async(prompt="Hello")
print(res)

Hello there!! How can i help you today buddy?


* Instead of passing a prompt we can also pass a complete conversation

In [48]:
messages= [
    {"role": "user", "content": "Hey there!"}
]
res= await rails.generate_async(messages=messages)
print(res)

{'role': 'assistant', 'content': 'Hello there!! How can i help you today buddy?'}


## Variables in combination with if else statements
* We can add personal gretings as well

In [49]:
colang_content = """
define user express greeting
    "hello"
    "hi"
    "yo"

define bot express greeting
    "Hello Boss! How can i help you today?"

define bot personal greeting
    "Hello $username, nice to see you again!"

define flow hello
    user express greeting
    
    if $username:
        bot personal greeting
    else:
        bot express greeting
    
"""

config= RailsConfig.from_content(
    yaml_content=yaml_content,
    colang_content=colang_content
)
rails= LLMRails(config=config)

In [50]:
messages=[
    {"role":"user", "content":"Hey there!"}
]
res= await rails.generate_async(messages=messages)
pprint(res)

{'content': 'Hello Boss! How can i help you today?', 'role': 'assistant'}


* Now instead of role: user we will add context and then we will add username to that

In [51]:
messages=[
    {"role":"context", "content":{"username":"Vicky"}},
    {"role":"user", "content":"Hey there!"}
]
res= await rails.generate_async(messages=messages)
pprint(res)

{'content': 'Hello Vicky, nice to see you again!', 'role': 'assistant'}


## Yeah!!!
### Now lets integrate with LangChain
* Besides normal wishing flow we are using another flow which means, if the first flow doesn't catch the user then second flow will be executed
* We are passing latest message to qa chain. So we have new method qa_chain and we have to use it and execute a function with execute keyword

In [52]:
colang_content="""
define user express greeting
    "hello"
    "hi"
    "yo"

define bot express greeting
    "Hello Buddy! How can i help you today?"

define bot personal greeting
    "Hello $username,Welcome back Sir!"

define flow hello
    user express greeting
    if $username
        bot express greeting
    else
        bot personal greeting

define flow
    user ...
    $answer = execute qa_chain(query=$last_user_message)
    bot $answer
"""
config = RailsConfig.from_content(
    yaml_content= yaml_content,
    colang_content= colang_content
)
rails= LLMRails(config= config)

## We will perfom this from vector database.

In [53]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

vectorstore= Chroma.from_texts(["CatBank's Furry Paws Credit Card offers 2% cashback on pet stores and 1% on all other purchases.",
                                 "Meow Loans at CatBank come with a competitive 5% APR, subject to your catnip credit score.",
                                 "Earn up to 4% interest with CatBank's Kitty Savers account, with a minimum deposit of $500.",
                                 "Invest in the Whisker Growth Fund with a minimum of $1,000 and enjoy purr-tential annual returns of 7%.",
                                 "Open a Feline Secure savings account with just $100; enjoy up to 3% interest with no monthly fees."],
                               embedding= OpenAIEmbeddings()
                               
                               
                               )
retriever= vectorstore.as_retriever()

### Create Prompts and retrievalQA

In [54]:
prompt_template=""" 
You are a helpful bot for our Bank. Answer if you have got content in "Context".
Otherwise tell the user in a friendly way that you do not know and can not help with that.

context: {context}

Question: {question}

Answer here:
"""

prompt= PromptTemplate(
    template= prompt_template,
    input_variables=["context", "question"]
)
chain_type_kwargs= {"prompt":prompt}

##Qa retriever
qa= RetrievalQA.from_chain_type(
    llm= rails.llm,
    chain_type="stuff",
    retriever= retriever,
    chain_type_kwargs= chain_type_kwargs
    
)

### You can register function to rails
* here we are registering RetrievalQA method to the rails

In [55]:
rails.register_action(qa, name="qa_chain")

In [56]:
history=[ {"role":"user", "content":"What is the minimum deposit amount of Kitty servers account?"}]
bot_message= await rails.generate_async(messages=history)
pprint(bot_message)

{'content': "The minimum deposit amount for CatBank's Kitty Savers account is "
            '$500.',
 'role': 'assistant'}


### Lets give some out of context question

In [57]:
history=[{"role":"user",
          "content":"any movie recommendations?"}]
bot_message= await rails.generate_async(messages=history)
pprint(bot_message)

{'content': "I'm sorry, I do not have any information on movie "
            'recommendations. But I can help you with any banking-related '
            'questions or services. Let me know how I can assist you with '
            'that!',
 'role': 'assistant'}


### Now lets create a guardrail for the context

In [58]:
new_colang_content= """
define user express greeting
    "hello"
    "hi" 
    "yo" 

define bot express greeting
    "Hello Buddy! How can i Help you?"
    "Hello Sir, How are you today? How can i help you?"

define bot personal greeting
    "Hello $username, Welcome Sir!"
    "Yo $username, Wasup!!"

define flow hello
    user express greeting
    if $username
        bot personal greeting
    else
        bot express greeting

define user ask about dogs
    "Can i get loan for my dog?"
    "what about dog credit cards?"

define bot no dog policy
    "Sir, This is a cat bank and we do not entertain dogs here. Hope you understand"
    "As a matter of fact that dog eats cats, we restrict them creating any accounts for them here."

define flow dog policy
    user ask about dogs
    bot no dog policy

define user ask silly questions
    "Can a horse create account?"
    "Can my cat directly walkin and create account?"
    "Can a cat take pension?"

define bot respond to silly question
    "We will consider this as funny, but only humans can perform these actions on behalf of cats and only for cats"

define flow silly cat questions
    user ask silly questions
    bot respond to silly question

define user express chit chat
    "what are your thoughts about current world situation?"
    "How do you deal with stress in your life?"
    "Are you in love with someone?"

define bot respond to chit chat
    "$username, We are here to talk about Cat banking, its better we stick to the topic and not deviate"
    "Well $username, I cannot answer about that may be someother time"

define flow chit chat
    user express chit chat
    bot responds to chit chat

define flow
    user ...
    $answer = execute qa_chain(query=$last_user_message)
    bot $answer
"""

In [59]:
config= RailsConfig.from_content(
    yaml_content= yaml_content,
    colang_content= new_colang_content
)
rails= LLMRails(config= config)
## register retrievalQA
rails.register_action(qa, name="qa_chain")

In [60]:
messages=[
    {"role":"context", "content":{"username":"Vicky"}},
    {"role":"user", "content":"Hello"},
]
res= await rails.generate_async(messages= messages)
pprint(res)

{'content': 'Hello Vicky, Welcome Sir!', 'role': 'assistant'}


In [61]:
messages=[
    {"role":"user", "content":"Can a pig create account?"},
]
res= await rails.generate_async(messages= messages)
pprint(res)

{'content': 'We will consider this as funny, but only humans can perform these '
            'actions on behalf of cats and only for cats',
 'role': 'assistant'}


In [62]:
messages=[
    {"role":"user", "content":"Is it possible to create an account for a horse?"},
]
res= await rails.generate_async(messages= messages)
pprint(res)

{'content': "I'm sorry, I am a bot for a Bank and I do not have information on "
            'creating an account for a horse. I recommend contacting a bank '
            'representative for assistance with that inquiry.',
 'role': 'assistant'}


## Subflows:
* We can use subv flows i.e., flows inside flows
* When you call a subflow you call with "do (do greet authentication)" parameter

In [63]:
subflow_colang_content="""
define user express greeting
    "Hi"
    "hello"
    "yo!"

define bot express greeting_unauthenticated
    "Welcome to CatBank, Please login or create a new account"

define bot express greeting_authenticated
    "Welcome back to CatBank, $username! How can we help you today?"

define subflow greeting with auth
    if $user_auth
        bot express greeting_authenticated
    else
        bot express greeting_unauthenticated

define flow hello
    user express greeting
    do greeting with auth
"""

In [64]:
config= RailsConfig.from_content(
    yaml_content= yaml_content,
    colang_content= subflow_colang_content
)
rails= LLMRails(config= config)

Message without authentication

In [65]:
messages=[
    {"role":"context", "content":{"user_auth":False, "username":"Vicky"}},
    {"role":"user", "content":"Hello There!"}
]
bot_response= await rails.generate_async(messages= messages)
pprint(bot_response)

{'content': 'Welcome to CatBank, Please login or create a new account',
 'role': 'assistant'}


With authentication: user_auth=True

In [66]:
messages=[
    {"role":"context", "content":{"user_auth":True, "username":"Vicky"}},
    {"role":"user", "content":"Hello There!"}
]
bot_response= await rails.generate_async(messages= messages)
pprint(bot_response)

{'content': 'Welcome back to CatBank, Vicky! How can we help you today?',
 'role': 'assistant'}


### Create the same using Config files


In [67]:
config= RailsConfig.from_path("config")
rails= LLMRails(config= config)

In [74]:
prompt_template=""" 
You are a helpful bot for our Bank. Answer if you have got content in "Context".
Otherwise tell the user in a friendly way that you do not know and can not help with that.

context: {context}

Question: {question}

Answer here:
"""


prompt= PromptTemplate(
    template= prompt_template,
    input_variables=["context", "question"]
)
chain_type_kwargs= {"prompt":prompt}

##Qa retriever
qa= RetrievalQA.from_chain_type(
    llm= rails.llm,
    chain_type="stuff",
    retriever= retriever,
    chain_type_kwargs= chain_type_kwargs
    
)

In [75]:
rails.register_action(qa, name="qa_chain")

In [70]:
message=[{"role":"user", "content":"Hello There!"}]
bot_response= await rails.generate_async(messages=message)
print(bot_response)

{'role': 'assistant', 'content': 'Hello Buddy! How can i Help you?'}


In [71]:

from langchain_core.output_parsers import StrOutputParser

In [72]:
message=[{"role":"user", "content":"Hello There!"}]
bot_response= rails.generate(messages=message)
print(bot_response)

{'role': 'assistant', 'content': 'Hello Buddy! How can i Help you?'}


In [78]:
message=[{"role":"user", "content":"How to make Tea?"}]
bot_response= rails.generate(messages=message)
print(bot_response)

Error while execution qa_chain: Missing some input keys: {'query'}


{'role': 'assistant', 'content': "I'm sorry, an internal error has occurred."}
